In [1]:
import numpy as np
import pandas as pd
import os,sys,cv2
import copy
import keras
sys.path.append(os.pardir)
from KerasGANs.Keras_StyleGAN import *
import matplotlib.pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [2]:
from keras.backend import tensorflow_backend as K

cf = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        allow_growth=True
    )
)
session = tf.Session(config=cf)
K.set_session(session)

## Config
基本いじらんでよい  
Target / GANのresが224以下の時はそれ以下に編集

In [20]:
VGG_INPUT = 224

## Setting
target_image_path : Embする画像のパス  
weights_path      : Embする空間をもつGANのweight(Keras)パス  
save_image_path   : 途中生成の画像を保存するパス  
save_image_name   : 途中生成の画像の名前  
res               : GANの出力サイズ  
stage             : GANのStageの数(1024だったら18 / 512だったら16)

In [4]:
target_image_path   = "IMG_2791.jpg"
weights_path        = "-----.h5"
save_image_path     = "IMG_2791/"
save_image_name     = "IMG_2791"
save_latent_path    = "IMG_2791_dlatent.npy"
res = 1024
stage = 18

## Prepare Model

In [21]:
##############################################
##        Prepare dlatent be trainable      ##
##############################################
def dlatent_model(init):
    def myinit(shape, dtype=None):
        return init[np.newaxis]
        
    inp = Input((1,))
    output = [LeakyReLU(0.2)(Dense(512,use_bias=False,name = "w_%i"%(i),kernel_initializer=myinit)(inp)) for i in range(stage)]
    output = Lambda(lambda x: tf.reshape(x,(-1,stage,512)))(output)
    return Model(inp,output)

##############################################
##        Prepare VGG model                  ##
##############################################
def VGG_model():
    input_tensor = Input(shape=(VGG_INPUT, VGG_INPUT, 3))
    VGG = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)
    out1 = Flatten()(VGG.get_layer("block1_conv1").output)
    out2 = Flatten()(VGG.get_layer("block1_conv2").output)
    out3 = Flatten()(VGG.get_layer("block2_conv2").output)
    out4 = Flatten()(VGG.get_layer("block3_conv3").output)
    out  = Concatenate()([out1,out2,out3,out4]) 

    return Model(input_tensor,out)

##############################################
## Prepare Image Generator(Style Based Gan) ##
##############################################
GAN = StyleGAN(output_res = res)
GAN.s_model.load_weights(weights_path)

# Instance
w_model = dlatent_model(np.random.normal(size=(512)))
g_model = GAN.synthesis_model()
v_model = VGG_model()


# Define Model
noise_inp     = [Input((g_model.input[1:][i].shape[1:])) for i in range(len(g_model.input)-1)]
inp           = Input((1,))
noise_inp     = [Input((g_model.input[1:][i].shape[1:])) for i in range(len(g_model.input)-1)]
embedding_w   = w_model(inp)
Generated_img = g_model([embedding_w] + noise_inp)
Reshape_img   = Lambda(lambda x: tf.image.resize(x,(res,res)))(Generated_img)
vgg_feature   = v_model(Reshape_img)
model = Model([inp]+noise_inp , [vgg_feature,Generated_img])


## Define In/Out

In [22]:
# Prepare Target
img = cv2.imread(target_image_path)[:,:,::-1]
t_img = cv2.resize(img,(res,res))
img = cv2.resize(img,(VGG_INPUT,VGG_INPUT))                                                          #resize_img
target_feature = v_model.predict(preprocess_input(copy.deepcopy(img)[np.newaxis]))           #F

In [13]:
# Prepare Input
one  = np.ones((1,))
noise = [np.random.normal(size=(1,w,w,1)) for w in [2**(int(i/2)+2) for i in range(stage)]]

In [14]:
def rmse(y_true,y_label):
    return tf.sqrt(tf.reduce_mean((y_true - y_label)**2))

## Model Compile

In [23]:
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.00000001, decay=0.0, amsgrad=False)
model.compile(adam,[rmse,rmse],loss_weights=[0.00001,0.00001])

## Training

In [25]:
iteration = 5000
for i in range(iteration):
    model.train_on_batch(x=[one]+noise,y=[target_feature,t_img[np.newaxis]])
    if i%100 == 99:
        print("Result epochs%i : "%(i+1),end="")
        print(model.evaluate(x=[one]+noise,y=[target_feature,t_img[np.newaxis]],verbose=0))
        img = model.predict([one]+noise)[1][0][:,:,::-1]
        img = np.array(np.clip(img,0,255),dtype="uint8")
        cv2.imwrite(save_image_path + save_image_name +"_%05i.jpg"%(i+1),img)
pred = w_model.predict(one)
np.save(save_latent_path,pred[0,0])

Result epochs100 : [0.004115925636142492, 331.44970703125, 80.14288330078125]
Result epochs200 : [0.0038880875799804926, 326.60699462890625, 62.2017822265625]
Result epochs300 : [0.0037314353976398706, 324.468994140625, 48.67454528808594]
Result epochs400 : [0.0037270979955792427, 321.3507995605469, 51.3590202331543]
Result epochs500 : [0.0035953596234321594, 317.2538757324219, 42.282108306884766]
Result epochs600 : [0.0035848510451614857, 312.4703369140625, 46.0147705078125]
Result epochs700 : [0.003500476013869047, 309.8905029296875, 40.1571044921875]
Result epochs800 : [0.003449872136116028, 307.5269775390625, 37.460227966308594]
Result epochs900 : [0.003370601451024413, 303.17022705078125, 33.88991165161133]
Result epochs1000 : [0.00330721284262836, 301.14239501953125, 29.578876495361328]
Result epochs1100 : [0.003296144073829055, 299.3504638671875, 30.26395034790039]
Result epochs1200 : [0.0032838478218764067, 299.3710632324219, 29.01374053955078]
Result epochs1300 : [0.0032501253